In [1]:
import pandas as pd
import numpy as np
import re
import recordlinkage
import pickle
import matplotlib.pyplot as plt
import Algorithmia

In [2]:
with open('../Data/all_muckrack_links.pkl', 'rb') as f:
    authors = pickle.load(f)
    
with open('../Data/news.pkl', 'rb') as f:
    news = pickle.load(f)
    
emails = pd.read_csv("../Data/compression_test.txt.gz", compression='gzip',sep = '\t')
emails = emails.fillna("")
emails = emails[['email_address', 'first_name_df', 'second_name_df']]

In [3]:
def first_name(url):
    return url[21:].split('-')[0]
first_name('https://muckrack.com/rafael-olmeda')

'rafael'

In [4]:
def author_outlet(author):
    return news[news.author == author].site_name.value_counts().index[0]

In [5]:
def concatenate_fields(author, outlet):
    return author + ' ' + outlet

In [6]:
authors['first_name_df'] = authors.apply(lambda x: first_name(x['request_url']), axis=1)

In [7]:
authors['common_outlet'] = authors.apply(lambda x: author_outlet(x['author']), axis=1)

In [8]:
authors['author + outlet'] = authors.apply(lambda x: concatenate_fields(x['author'], x['common_outlet']), axis=1)

In [9]:
authors

,author,article_count,request_url,redirect_url,method,confidence,first_name_df,common_outlet,author + outlet
0,Neil Shaw,523,https://muckrack.com/neil-shaw,https://muckrack.com/neil-shaw-1,brute force,-,neil,WalesOnline,Neil Shaw WalesOnline
1,James Rodger,512,https://muckrack.com/james-rodger,https://muckrack.com/james-rodger,brute force,-,james,BirminghamLive,James Rodger BirminghamLive
2,Jack Davis,251,https://muckrack.com/jack-davis,https://muckrack.com/jack-davis,brute force,-,jack,The Western Journal,Jack Davis The Western Journal
3,Adam Wells,223,https://muckrack.com/adam-wells,https://muckrack.com/adam-wells,brute force,-,adam,Bleacher Report,Adam Wells Bleacher Report
4,Sophie McCoid,218,https://muckrack.com/sophie-mccoid,https://muckrack.com/sophie-mccoid,brute force,-,sophie,Liverpool Echo,Sophie McCoid Liverpool Echo
...,...,...,...,...,...,...,...,...,...
3833,Levi Sumagaysay,10,https://muckrack.com/levi-sumagaysay,https://muckrack.com/levi-sumagaysay,brute force,-,levi,MarketWatch,Levi Sumagaysay MarketWatch
3834,Ben Ashford,10,https://muckrack.com/ben-ashford,https://muckrack.com/ben-ashford,brute force,-,ben,Mail Online,Ben Ashford Mail Online
3835,Craig Brown,10,https://muckrack.com/craig-brown,https://muckrack.com/craig-brown,brute force,-,craig,Mail Online,Craig Brown Mail Online
3836,Courtney Connley,10,https://muckrack.com/courtney-connley,https://muckrack.com/courtney-connley,brute force,-,courtney,CNBC,Courtney Connley CNBC


In [10]:
indexer = recordlinkage.Index()
indexer.block('first_name_df')
candidate_links = indexer.index(authors, emails)
print("candidate_links:", len(candidate_links))

candidate_links: 1296083


In [11]:
emails

,email_address,first_name_df,second_name_df
0,rating@mail.ru,,
1,xb3@mail.ru,,
2,info@aktiencheck.de,,
3,legals@heartmediact.com,,
4,8b4e078a51d04e0e9efdf470027f0ec1@sentry.wixpre...,,
...,...,...,...
2574181,hyzhenhua@mail.tzptt.zj.cn,,
2574182,hyzgkxni@gmail.com,,
2574183,hyzbhtbq@ygfqvw.com,,
2574184,hyzb@uibe.edu.cn,,


In [12]:
compare_cl = recordlinkage.Compare()
compare_cl.string('author + outlet', 'email_address', label = 'match')
features = compare_cl.compute(candidate_links, authors, emails)

In [13]:
features

match
0    7751     0.222222
     11897    0.222222
     28036    0.142857
     30146    0.142857
     58595    0.206897
...                ...
3833 2194924  0.161290
     2194925  0.111111
     2194926  0.222222
     2194927  0.222222
     2194942  0.148148

[1296083 rows x 1 columns]

In [14]:
author_indexes = set()
for i in features[features.match >= 0.5].index:
    author_indexes.add(i[0])

In [15]:
data = features[features.match >= 0.5].sort_index(level = [0, 1], sort_remaining = False)

In [16]:
data

match
1    2080352  0.551724
11   205311   0.500000
     314799   0.500000
33   38933    0.533333
     947087   0.533333
...                ...
3816 438843   0.600000
     1598269  0.560000
3830 687117   0.500000
3832 34788    0.512821
3833 152891   0.677419

[1842 rows x 1 columns]

In [17]:
data.loc[33].sort_values('match', ascending = False).head(1)

,match
38933,0.533333


In [18]:
matches = pd.DataFrame (columns = ['author_index', 'email_index', 'match_threshold', 'found_url'])

In [19]:
matches

,author_index,email_index,match_threshold,found_url


In [20]:
for i in author_indexes:
    temp = data.loc[i].sort_values('match', ascending = False).head(1)
    row = pd.Series(list([i, temp.index[0], round(temp.match.values[0], 3), '-']), index = matches.columns)
    matches = matches.append(row, ignore_index = True) 

In [21]:
matches[matches.author_index == 3816]

,author_index,email_index,match_threshold,found_url
386,3816,438843,0.6,-


In [22]:
client = Algorithmia.client('simVvn80bx8D0mbOH5JC8nk6hb11')
algo = client.algo('specrom/Google_scraper/0.1.4')
algo.set_options(timeout=300) # optional

In [23]:
matches

,author_index,email_index,match_threshold,found_url
0,1,2080352,0.552,-
1,2054,1764154,0.560,-
2,11,205311,0.500,-
3,2061,2958,0.548,-
4,2074,312243,0.615,-
...,...,...,...,...
408,1989,6443,0.538,-
409,2011,785848,0.581,-
410,2022,809326,0.625,-
411,2023,5612,0.528,-


In [36]:
for i in range(len(matches)):
    print()
    print(authors.iloc[int(matches.iloc[i].author_index)].method)
    if authors.iloc[int(matches.iloc[i].author_index)].method == 'no match':
        matches.loc[i, 'found_url'] = 'invalid'
        print("not found", i)
    else:
        original_link = authors.iloc[int(matches.iloc[i].author_index)].redirect_url
        email = emails.iloc[int(matches.iloc[i].email_index)].email_address
        query = {"query": f"{email} muckrack"}
        results = algo.pipe(query).result
        if results and original_link == results[0]['url']:
            matches.loc[i, 'found_url'] = 'yes'
            print("found", i, email, original_link, results[0]['url'])
        else:
            matches.loc[i, 'found_url'] = 'no'
            print("not found", i, email, original_link)


brute force
not found 0 james.fowler@birminghamal.gov https://muckrack.com/james-rodger

brute force
not found 1 david.gallina@laerdal.com https://muckrack.com/david-hall

brute force
not found 2 jenna.ciccotelli@globe.com https://muckrack.com/jenna-ciccotelli

brute force
not found 3 anthony.colangelo@theage.com.au https://muckrack.com/anthony-colangelo

brute force
not found 4 paul.doyle@theguardian.com https://muckrack.com/paul-doyle

tfidf match
not found 5 matthew.cooper@latimes.com https://muckrack.com/matthew-cooper-3

brute force
not found 6 helen.barnes@citypress.co.uk https://muckrack.com/helen-kelly

brute force
not found 7 stuart.patterson@the-sun.co.uk https://muckrack.com/stuart-ballard

brute force
not found 8 claire.norman@foe.co.uk https://muckrack.com/claire-anderson

brute force
not found 9 beth.mishler-elmore@heavy.com https://muckrack.com/beth-mishler-elmore

brute force
not found 10 sophie.portas@faber.co.uk https://muckrack.com/sophie-harris

brute force
not fou

not found 93 david.fanner@theguardian.com https://muckrack.com/david-hytner

brute force
not found 94 karen.kucher@sduniontribune.com https://muckrack.com/karen-caffarini

brute force
not found 95 chelsia.marcius@thedailybeast.com https://muckrack.com/chelsia-marcius

brute force
not found 96 sarah.bradbury@remax.net https://muckrack.com/sarah-bradbury

brute force
not found 97 alex.kaufman@canaries.co.uk. https://muckrack.com/alex-shipman

brute force
not found 98 enrique.lanuza@uv.es https://muckrack.com/enrique-dans

brute force
not found 99 mark.duell@mailonline.co.uk https://muckrack.com/mark-duell

brute force
not found 100 james.robinson@mailonline.co.uk https://muckrack.com/james-robinson

tfidf match
not found 101 lisa.paulashouse@gmail.com https://muckrack.com/lisa-jhung

brute force
not found 102 beth.heyn@heavy.com. https://muckrack.com/beth-heyn

brute force
not found 103 anthony.armor@gmail.com https://muckrack.com/anthony-salamone

tfidf match
not found 104 douglas.busvi

not found 181 christine.garcia@state.co.us https://muckrack.com/christine-younan

brute force
not found 182 megan.ryan@startribune.com https://muckrack.com/megan-jones

brute force
not found 183 maxine.shen@mailonline.com https://muckrack.com/maxine-shen

tfidf match
not found 184 stephen.robertson@myhorrynews.com https://muckrack.com/stephen-brown

brute force
not found 185 elizabeth.mcknight@utoledo.edu https://muckrack.com/eknightofficial

brute force
not found 186 alexander.brockmann@dlv.de https://muckrack.com/alexander-brock

brute force
found 187 ali.daher@mailonline.com https://muckrack.com/aliiscoollike https://muckrack.com/aliiscoollike

brute force
not found 188 trevor.hunnicutt@tr.com https://muckrack.com/trhunnicutt

brute force
found 189 shannon.ayin@chicagotribune.com https://muckrack.com/shannon-ryan https://muckrack.com/shannon-ryan

brute force
not found 190 darren.boyle@mailonline.co.uk https://muckrack.com/darren-boyle

brute force
not found 191 michael.fowler@theag

AlgorithmException: "Account doesn't have any remaining credits"

In [34]:
matches.iloc[105], authors.iloc[372], emails.iloc[514138]

(author_index          372
 email_index        514138
 match_threshold       0.5
 found_url              no
 Name: 105, dtype: object,
 author                                  Sam Baker
 article_count                                  71
 request_url        https://muckrack.com/sam-baker
 redirect_url       https://muckrack.com/sam-baker
 method                                brute force
 confidence                                      -
 first_name_df                                 sam
 common_outlet                         Mail Online
 author + outlet             Sam Baker Mail Online
 Name: 372, dtype: object,
 email_address     sam.baker@mailonline.co.uk
 first_name_df                            sam
 second_name_df                         baker
 Name: 514138, dtype: object)

In [39]:
matches[matches.found_url == 'yes']

,author_index,email_index,match_threshold,found_url
117,2472,20379,0.558,yes
123,2510,94245,0.667,yes
127,2520,169345,0.609,yes
131,2534,314073,0.846,yes
139,508,437904,0.545,yes
141,2573,1356164,0.600,yes
143,2590,36229,0.750,yes
153,584,339648,0.560,yes
179,2759,914495,0.688,yes
187,739,753028,0.542,yes


In [26]:
query = {"query": f"soumyaparekh"}
results = algo.pipe(query).result

In [27]:
results

[]

In [ ]:
authors.iloc[int(matches.iloc[11].author_index)].method

In [ ]:
authors.method.value_counts()

In [ ]:
matches.found_url.value_counts()

In [ ]:
results[0]['url']

In [ ]:
link